# Forumla1 Can we predict who is going to be on pole?

The aim of the project: 
The aim of the project is build a model which can predict formula qualifying position based on characteristics of the driver, car and circuit.

### Data Available for this project:
- Ergast API database historical result data from 1950 to present day
- Weather data from Visual Crossings API
- FastF1 api telemetry data from 2018 to present day

This notebook will analyse and clean the Ergast Database

Ergast Data structure:
<div ><img style="height:500px"   src ='./images/ergast_db.png' /> </div>




In [2]:
import pandas as pd
import numpy as np
import sys
from scripts import ds_ultils,data_cleaning, f1_ultils

import os
import requests
import re

Run python scripts to collect weather data from webscrapping and api

In [2]:
## %cd ./scripts/
## %run Weather_data_collection.py


In [3]:
f1_data=f1_ultils.read_ergast_files('./data/raw') # help function load each csv and save to dictionary

In [4]:
for key in f1_data.keys():
    print(f"Table {key} :")
    ds_ultils.df_explore(f1_data[key])

Table df_circuits :

Dataframe Rows: 76 	 Dataframe Columns: 9


Summary of the dataframe : 
  
 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76 entries, 0 to 75
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   circuitId   76 non-null     int64  
 1   circuitRef  76 non-null     object 
 2   name        76 non-null     object 
 3   location    76 non-null     object 
 4   country     76 non-null     object 
 5   lat         76 non-null     float64
 6   lng         76 non-null     float64
 7   alt         76 non-null     object 
 8   url         76 non-null     object 
dtypes: float64(2), int64(1), object(6)
memory usage: 5.5+ KB
None
-------------

Total null rows: 0
Percentage null rows: 0.0%

Total duplicate rows: 0
Percentage dupe rows: 0.0%
------------



,circuitId,circuitRef,name,location,country,lat,lng,alt,url
0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.96800,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
1,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.73800,18,http://en.wikipedia.org/wiki/Sepang_Internatio...
2,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.51060,7,http://en.wikipedia.org/wiki/Bahrain_Internati...
3,4,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.57000,2.26111,109,http://en.wikipedia.org/wiki/Circuit_de_Barcel...
4,5,istanbul,Istanbul Park,Istanbul,Turkey,40.95170,29.40500,130,http://en.wikipedia.org/wiki/Istanbul_Park


------------------------------

Table df_constructors :

Dataframe Rows: 211 	 Dataframe Columns: 5


Summary of the dataframe : 
  
 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211 entries, 0 to 210
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   constructorId   211 non-null    int64 
 1   constructorRef  211 non-null    object
 2   name            211 non-null    object
 3   nationality     211 non-null    object
 4   url             211 non-null    object
dtypes: int64(1), object(4)
memory usage: 8.4+ KB
None
-------------

Total null rows: 0
Percentage null rows: 0.0%

Total duplicate rows: 0
Percentage dupe rows: 0.0%
------------



,constructorId,constructorRef,name,nationality,url
0,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren
1,2,bmw_sauber,BMW Sauber,German,http://en.wikipedia.org/wiki/BMW_Sauber
2,3,williams,Williams,British,http://en.wikipedia.org/wiki/Williams_Grand_Pr...
3,4,renault,Renault,French,http://en.wikipedia.org/wiki/Renault_in_Formul...
4,5,toro_rosso,Toro Rosso,Italian,http://en.wikipedia.org/wiki/Scuderia_Toro_Rosso


------------------------------

Table df_constructor_results :

Dataframe Rows: 12110 	 Dataframe Columns: 5


Summary of the dataframe : 
  
 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12110 entries, 0 to 12109
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   constructorResultsId  12110 non-null  int64  
 1   raceId                12110 non-null  int64  
 2   constructorId         12110 non-null  int64  
 3   points                12110 non-null  float64
 4   status                12110 non-null  object 
dtypes: float64(1), int64(3), object(1)
memory usage: 473.2+ KB
None
-------------

Total null rows: 0
Percentage null rows: 0.0%

Total duplicate rows: 0
Percentage dupe rows: 0.0%
------------



...

------------------------------

Table df_sprint_results :

Dataframe Rows: 100 	 Dataframe Columns: 16


Summary of the dataframe : 
  
 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   resultId        100 non-null    int64 
 1   raceId          100 non-null    int64 
 2   driverId        100 non-null    int64 
 3   constructorId   100 non-null    int64 
 4   number          100 non-null    int64 
 5   grid            100 non-null    int64 
 6   position        100 non-null    object
 7   positionText    100 non-null    object
 8   positionOrder   100 non-null    int64 
 9   points          100 non-null    int64 
 10  laps            100 non-null    int64 
 11  time            100 non-null    object
 12  milliseconds    100 non-null    object
 13  fastestLap      100 non-null    object
 14  fastestLapTime  100 non-null    object
 15  statu

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,fastestLapTime,statusId
0,1,1061,830,9,33,2,1,1,1,3,17,25:38.426,1538426,14,1:30.013,1
1,2,1061,1,131,44,1,2,2,2,2,17,+1.430,1539856,17,1:29.937,1
2,3,1061,822,131,77,3,3,3,3,1,17,+7.502,1545928,17,1:29.958,1
3,4,1061,844,6,16,4,4,4,4,0,17,+11.278,1549704,16,1:30.163,1
4,5,1061,846,1,4,6,5,5,5,0,17,+24.111,1562537,16,1:30.566,1


------------------------------

Table df_status :

Dataframe Rows: 139 	 Dataframe Columns: 2


Summary of the dataframe : 
  
 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139 entries, 0 to 138
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   statusId  139 non-null    int64 
 1   status    139 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.3+ KB
None
-------------

Total null rows: 0
Percentage null rows: 0.0%

Total duplicate rows: 0
Percentage dupe rows: 0.0%
------------



,statusId,status
0,1,Finished
1,2,Disqualified
2,3,Accident
3,4,Collision
4,5,Engine


------------------------------



Files often have "\N" instead of null or empty values. For each csv file, I will replace the "\N" with null values and re inspect the data

**Columns with significant amount of null values :**

In [5]:
for key in f1_data.keys():
    df = f1_data[key]
    df = data_cleaning.convert_string_tonan(df)
    #df.to_csv(f'../data/clean/{key}_clean.csv',index=False)
    print(f'Dataframe {key}')
    ds_ultils.nan_checker(f1_data[key])
    print('-----------')

Dataframe df_circuits
This dataframe has 2 NaN values
 The NaN values come from: ['alt']. 

-----------
Dataframe df_constructors
This dataframe has 0 NaN values
-----------
Dataframe df_constructor_results
This dataframe has 12093 NaN values
 The NaN values come from: ['status']. 
	 These ['status'] columns have > 50% NaNs
-----------
Dataframe df_constructor_standings
This dataframe has 0 NaN values
-----------
Dataframe df_drivers
This dataframe has 1560 NaN values
 The NaN values come from: ['number', 'code']. 
	 These ['number', 'code'] columns have > 50% NaNs
-----------
Dataframe df_driver_standings
This dataframe has 0 NaN values
-----------
Dataframe df_lap_times
This dataframe has 0 NaN values
-----------
Dataframe df_pit_stops
This dataframe has 0 NaN values
-----------
Dataframe df_qualifying
This dataframe has 10861 NaN values
 The NaN values come from: ['q1', 'q2', 'q3']. 
	 These ['q3'] columns have > 50% NaNs
-----------
Dataframe df_races
This dataframe has 11257 NaN v

In [6]:
df_circuits_clean = f1_data['df_circuits'].copy()
df_circuits_clean.head()

,circuitId,circuitRef,name,location,country,lat,lng,alt,url
0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.96800,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
1,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.73800,18,http://en.wikipedia.org/wiki/Sepang_Internatio...
2,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.51060,7,http://en.wikipedia.org/wiki/Bahrain_Internati...
3,4,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.57000,2.26111,109,http://en.wikipedia.org/wiki/Circuit_de_Barcel...
4,5,istanbul,Istanbul Park,Istanbul,Turkey,40.95170,29.40500,130,http://en.wikipedia.org/wiki/Istanbul_Park


In [7]:
## Clean up the two nan's in altitude

In [8]:
df_circuits_clean=data_cleaning.fill_altitude_nans(df_circuits_clean) # replace nan values of altitude using lat and long from api

In [9]:
df_circuits_clean[df_circuits_clean.isna().any(axis=1)]

,circuitId,circuitRef,name,location,country,lat,lng,alt,url


Altitude is filled in 

CircuitRef is a repitition of name, however for combination with telemetry data will keep it for now

Constructors database

In [10]:
df_constructors_clean =f1_data['df_constructors'] 

In [11]:
df_constructors_clean.head()

,constructorId,constructorRef,name,nationality,url
0,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren
1,2,bmw_sauber,BMW Sauber,German,http://en.wikipedia.org/wiki/BMW_Sauber
2,3,williams,Williams,British,http://en.wikipedia.org/wiki/Williams_Grand_Pr...
3,4,renault,Renault,French,http://en.wikipedia.org/wiki/Renault_in_Formul...
4,5,toro_rosso,Toro Rosso,Italian,http://en.wikipedia.org/wiki/Scuderia_Toro_Rosso


Same length of names for Constructor ref and name and duplicate of values. will drop name


In [12]:
df_constructors_clean.drop(columns='name',axis=0,inplace=True)

Constructor Results

In [13]:
df_constructor_results_clean=f1_data['df_constructor_results'].copy()

In [14]:
df_constructor_results_clean.head()

,constructorResultsId,raceId,constructorId,points,status
0,1,18,1,14.0,NaN
1,2,18,2,8.0,NaN
2,3,18,3,9.0,NaN
3,4,18,4,5.0,NaN
4,5,18,5,2.0,NaN


All status of constructors is nan - will drop status column for constructors 

In [15]:
df_constructor_results_clean.drop(columns='status',axis=0,inplace=True)

Constructor Standings 

In [16]:
df_constructor_standings_clean=f1_data['df_constructor_standings'].copy()

In [17]:
df_constructor_standings_clean.head()

,constructorStandingsId,raceId,constructorId,points,position,positionText,wins
0,1,18,1,14.0,1,1,1
1,2,18,2,8.0,3,3,0
2,3,18,3,9.0,2,2,0
3,4,18,4,5.0,4,4,0
4,5,18,5,2.0,5,5,0


In [18]:
df_constructor_standings_clean[df_constructor_standings_clean['positionText']=='E']

,constructorStandingsId,raceId,constructorId,points,position,positionText,wins
217,25756,38,1,44.0,11,E,1
227,25767,39,1,58.0,11,E,1
236,25778,40,1,76.0,11,E,2
245,25789,41,1,88.0,11,E,3
254,25800,42,1,106.0,11,E,4
263,25811,43,1,114.0,11,E,4
272,25822,44,1,128.0,11,E,5
281,25833,45,1,138.0,11,E,5
290,25844,46,1,138.0,11,E,6
299,25855,47,1,148.0,11,E,6


In [19]:
df_constructors_clean[df_constructors_clean['constructorId']==1]

,constructorId,constructorRef,nationality,url
0,1,mclaren,British,http://en.wikipedia.org/wiki/McLaren


In [20]:
df_races_weather_all_clean=f1_data['df_races_weather_all'].copy()

In [21]:
df_races_weather_all_clean[df_races_weather_all_clean['raceId']==36]

,Unnamed: 0,raceId,year,circuitId,name,date,time,url,quali_date,quali_time,...,lng,race_temp,race_precip,race_humidity,race_condition,quali_temp,quali_precip,quali_humidity,quali_condition,race_weather_wiki
2,2,36,2007,1,Australian Grand Prix,2007-03-18,03:00:00,http://en.wikipedia.org/wiki/2007_Australian_G...,NaN,NaN,...,144.968,11.05,0.0,79.9525,Partially cloudy,12.925,0.0,82.99,Clear,Weather\nClear with temperatures reaching up t...


Maclearn were excluded from the constructors standings in 2007 due to possessing confidential information from the Ferrari team.
https://thefacts.app/facts/events/the-mclaren-f1-team-are-found-guilty-of-possessing-confidential-information-from-the-ferrari-team-fined-100-million-and-excluded-from-the-constructors-championship-standings

For our model, we will include there position and use position and not PositionText. Since it is just Maclearn


In [22]:
df_constructor_standings_clean.drop(columns='positionText',axis=0, inplace = True)

Drivers database

In [23]:
df_drivers_clean=f1_data['df_drivers'].copy()
df_drivers_clean.head()

,driverId,driverRef,number,code,forename,surname,dob,nationality,url
0,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
1,2,heidfeld,NaN,HEI,Nick,Heidfeld,1977-05-10,German,http://en.wikipedia.org/wiki/Nick_Heidfeld
2,3,rosberg,6,ROS,Nico,Rosberg,1985-06-27,German,http://en.wikipedia.org/wiki/Nico_Rosberg
3,4,alonso,14,ALO,Fernando,Alonso,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso
4,5,kovalainen,NaN,KOV,Heikki,Kovalainen,1981-10-19,Finnish,http://en.wikipedia.org/wiki/Heikki_Kovalainen


It has nan only in number and code. Number and code are repitions of driverref

There are no drivers with the same driverRef, therefore we can drop number and code. Forename and surname is the same as driverref. However there are non unique surnames, like Mike and Micheal Schumacher. Therefore we will drop forename and surname.

In [24]:
df_drivers_clean.drop(columns=['code','forename','surname'],axis=0,inplace=True)

Driver standings

In [25]:
df_driver_standings_clean=f1_data['df_driver_standings'].copy()

In [26]:
df_driver_standings_clean.head()

,driverStandingsId,raceId,driverId,points,position,positionText,wins
0,1,18,1,10.0,1,1,1
1,2,18,2,8.0,2,2,0
2,3,18,3,6.0,3,3,0
3,4,18,4,5.0,4,4,0
4,5,18,5,4.0,5,5,0


One disqualified result

In [27]:
df_driver_standings_clean[df_driver_standings_clean['positionText'] == 'D']

,driverStandingsId,raceId,driverId,points,position,positionText,wins
4375,11031,223,30,78.0,26,D,5


In [28]:
df_drivers_clean[df_drivers_clean['driverId'] == 30]

,driverId,driverRef,number,dob,nationality,url
29,30,michael_schumacher,NaN,1969-01-03,German,http://en.wikipedia.org/wiki/Michael_Schumacher


In [29]:
df_races_weather_all_clean[df_races_weather_all_clean['raceId']==223]

,Unnamed: 0,raceId,year,circuitId,name,date,time,url,quali_date,quali_time,...,lng,race_temp,race_precip,race_humidity,race_condition,quali_temp,quali_precip,quali_humidity,quali_condition,race_weather_wiki
754,758,223,1997,26,European Grand Prix,1997-10-26,NaN,http://en.wikipedia.org/wiki/1997_European_Gra...,NaN,NaN,...,-6.03417,21.5,0.0,73.85,Partially cloudy,20.9,0.0,70.28,Partially cloudy,Weather\nSunny with temperatures reaching up t...


Michael Schumacher was disqualified from 1997 F1 world championship for trying to take out Jacques Villeneuve https://sportslumo.com/motorsport/f1-rewind-when-michael-schumacher-was-disqualified-from-f1-world-championship-for-trying-to-take-out-jacques-villeneuve/

This was the end of the season the last race - hence only one occurence. Since we can not predict these "Black Swan" type events we will take his true position before exclusion

Therefore drop PositionText column as it is a repetition of position

In [30]:
df_driver_standings_clean.drop(columns='positionText',axis=0,inplace=True)

Lap times

In [31]:
df_lap_times_clean=f1_data['df_lap_times'].copy()

In [32]:
df_lap_times_clean.head()

,raceId,driverId,lap,position,time,milliseconds
0,841,20,1,1,1:38.109,98109
1,841,20,2,1,1:33.006,93006
2,841,20,3,1,1:32.713,92713
3,841,20,4,1,1:32.803,92803
4,841,20,5,1,1:32.342,92342


In [33]:
(df_lap_times_clean['time'].apply(f1_ultils.convert_time_miliseconds) != df_lap_times_clean['milliseconds']).sum()

0

Time is the same as milliseconds, drop time

In [34]:
df_lap_times_clean.drop(columns='time',axis=0,inplace=True)

df_pit_stops_clean - 0 nans

In [35]:
df_pit_stops_clean=f1_data['df_pit_stops'].copy()

In [36]:
df_pit_stops_clean.head()

,raceId,driverId,stop,lap,time,duration,milliseconds
0,841,153,1,1,17:05:23,26.898,26898
1,841,30,1,1,17:05:52,25.021,25021
2,841,17,1,11,17:20:48,23.426,23426
3,841,4,1,12,17:22:34,23.251,23251
4,841,13,1,13,17:24:10,23.842,23842


In [37]:
df_pit_stops_clean['duration'].dtype

dtype('O')

In [38]:
(df_pit_stops_clean['duration'].apply(f1_ultils.convert_time_miliseconds) != df_pit_stops_clean['milliseconds']).sum()

0

All duration = milliseconds. Therefore drop duration. Time is time of day, this could be combined with date if needed

In [39]:
df_pit_stops_clean.drop(columns='duration',axis=0,inplace=True)

df_qualifying_clean - nans in q

In [40]:
df_qualifying_clean=f1_data['df_qualifying'].copy()

In [41]:
df_qualifying_clean.head()

,qualifyId,raceId,driverId,constructorId,number,position,q1,q2,q3
0,1,18,1,1,22,1,1:26.572,1:25.187,1:26.714
1,2,18,9,2,4,2,1:26.103,1:25.315,1:26.869
2,3,18,5,1,23,3,1:25.664,1:25.452,1:27.079
3,4,18,13,6,2,4,1:25.994,1:25.691,1:27.178
4,5,18,2,2,3,5,1:25.960,1:25.518,1:27.236


Number column is driver number from ergast api, therefore can be dropped 

Nan's in qualifying results happen because only the top 15 progress to Q2 session and then only top 10 to Q3. However are there more Nan's that expected?

In [42]:
frames=[]
for race in df_qualifying_clean['raceId'].unique():
    df=pd.DataFrame({'raceId':race,
                     'q1_nans':df_qualifying_clean.loc[df_qualifying_clean['raceId']==race,'q1'].isna().sum(),
                     'q1_results':(~df_qualifying_clean.loc[df_qualifying_clean['raceId']==race,'q1'].isna()).sum(),                     
                     'q2_nans':df_qualifying_clean.loc[df_qualifying_clean['raceId']==race,'q2'].isna().sum(),
                     'q2_results':(~df_qualifying_clean.loc[df_qualifying_clean['raceId']==race,'q2'].isna()).sum(),
                     'q3_nans':df_qualifying_clean.loc[df_qualifying_clean['raceId']==race,'q3'].isna().sum(),
                     'q3_results':(~df_qualifying_clean.loc[df_qualifying_clean['raceId']==race,'q3'].isna()).sum(),
                     'num_driver': len(df_qualifying_clean.loc[df_qualifying_clean['raceId'] == race,'driverId'].unique())
                    },index=[0])
    frames.append(df)
    
df=pd.concat(frames,ignore_index=True)

In [43]:
df

,raceId,q1_nans,q1_results,q2_nans,q2_results,q3_nans,q3_results,num_driver
0,18,0,22,8,14,13,9,22
1,19,0,22,6,16,12,10,22
2,20,0,22,6,16,12,10,22
3,21,0,22,6,16,12,10,22
4,22,0,20,5,15,10,10,20
...,...,...,...,...,...,...,...,...
437,1085,0,20,5,15,12,8,20
438,1086,0,20,5,15,10,10,20
439,1087,0,20,5,15,10,10,20
440,1088,0,20,5,15,11,9,20


In [44]:
df['q3_check']=df['q3_nans']+df['q3_results']
df['q2_check']=df['q2_nans']+df['q2_results']

In [45]:
df.head()

,raceId,q1_nans,q1_results,q2_nans,q2_results,q3_nans,q3_results,num_driver,q3_check,q2_check
0,18,0,22,8,14,13,9,22,22,22
1,19,0,22,6,16,12,10,22,22,22
2,20,0,22,6,16,12,10,22,22,22
3,21,0,22,6,16,12,10,22,22,22
4,22,0,20,5,15,10,10,20,20,20


In [46]:
(~df['q3_check'] == df['num_driver']).sum()

0

In [47]:
(~df['q2_check'] == df['num_driver']).sum()

0

No the number of nan's is as expected

What about Q1 nan's everyone races in Q1 therefore should be 0

In [48]:
df_qualifying_clean[df_qualifying_clean['q1'].isna()]

,qualifyId,raceId,driverId,constructorId,number,position,q1,q2,q3
147,148,24,20,5,15,20,NaN,NaN,NaN
307,308,32,21,10,21,20,NaN,NaN,NaN
455,456,39,26,5,19,22,NaN,NaN,NaN
477,478,40,27,12,21,22,NaN,NaN,NaN
763,764,53,8,1,3,22,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
8994,9035,228,85,28,23,22,NaN,NaN,NaN
9174,9232,1075,852,213,22,20,NaN,NaN,NaN
9194,9252,1076,840,117,18,20,NaN,NaN,NaN
9214,9272,1077,848,3,23,20,NaN,NaN,NaN


In [49]:
df_qualifying_clean.loc[(df_qualifying_clean['q1'].isna()),'raceId' ].value_counts()[df_qualifying_clean.loc[(df_qualifying_clean['q1'].isna()),'raceId' ].value_counts() > 1]

256     24
84       4
88       3
90       3
96       3
87       2
91       2
93       2
108      2
998      2
1013     2
97       2
950      2
225      2
199      2
1026     2
105      2
71       2
1057     2
885      2
855      2
846      2
59       2
1059     2
123      2
910      2
106      2
82       2
Name: raceId, dtype: int64

In [50]:
df_qualifying_clean[df_qualifying_clean['raceId']==256]

,qualifyId,raceId,driverId,constructorId,number,position,q1,q2,q3
2568,2570,256,71,3,5,1,NaN,NaN,NaN
2569,2571,256,14,3,6,2,NaN,NaN,NaN
2570,2572,256,30,22,1,3,NaN,NaN,NaN
2571,2573,256,77,6,28,4,NaN,NaN,NaN
2572,2574,256,55,6,27,5,NaN,NaN,NaN
2573,2575,256,49,15,30,6,NaN,NaN,NaN
2574,2576,256,22,17,14,7,NaN,NaN,NaN
2575,2577,256,65,22,2,8,NaN,NaN,NaN
2576,2578,256,56,17,15,9,NaN,NaN,NaN
2577,2579,256,87,1,7,10,NaN,NaN,NaN


In [51]:
df_races_weather_all_clean[df_races_weather_all_clean['raceId'] == 256]

,Unnamed: 0,raceId,year,circuitId,name,date,time,url,quali_date,quali_time,...,lng,race_temp,race_precip,race_humidity,race_condition,quali_temp,quali_precip,quali_humidity,quali_condition,race_weather_wiki
776,780,256,1995,29,Australian Grand Prix,1995-11-12,NaN,http://en.wikipedia.org/wiki/1995_Australian_G...,NaN,NaN,...,138.617,18.9,0.0,40.29,Clear,14.4,0.02,60.51,"Rain, Partially cloudy",Weather\nSunny[2]


In [52]:
df_qualifying_clean[df_qualifying_clean['raceId']==84]

,qualifyId,raceId,driverId,constructorId,number,position,q1,q2,q3
1278,1279,84,8,1,9,1,1:26.797,NaN,NaN
1279,1280,84,21,4,6,2,1:27.039,NaN,NaN
1280,1281,84,4,4,5,3,1:27.050,NaN,NaN
1281,1282,84,31,1,10,4,1:27.352,NaN,NaN
1282,1283,84,15,7,16,5,1:27.501,NaN,NaN
1283,1284,84,2,3,8,6,1:27.929,NaN,NaN
1284,1285,84,17,3,7,7,1:27.944,NaN,NaN
1285,1286,84,13,15,12,8,1:28.419,NaN,NaN
1286,1287,84,23,7,17,9,1:28.594,NaN,NaN
1287,1288,84,32,9,15,10,1:28.963,NaN,NaN


In [53]:
df_races_weather_all_clean[df_races_weather_all_clean['raceId'] == 84]

,Unnamed: 0,raceId,year,circuitId,name,date,time,url,quali_date,quali_time,...,lng,race_temp,race_precip,race_humidity,race_condition,quali_temp,quali_precip,quali_humidity,quali_condition,race_weather_wiki
183,183,84,2005,5,Turkish Grand Prix,2005-08-21,15:00:00,http://en.wikipedia.org/wiki/2005_Turkish_Gran...,NaN,NaN,...,29.405,26.033333,0.0,56.603333,Partially cloudy,27.566667,0.0,43.206667,Partially cloudy,Weather\nSunny; 30°C Air; 45°C Track


In [54]:
df_qualifying_clean[df_qualifying_clean['raceId']==88]

,qualifyId,raceId,driverId,constructorId,number,position,q1,q2,q3
1358,1359,88,23,7,17,1,1:46.106,NaN,NaN
1359,1360,88,18,16,3,2,1:46.141,NaN,NaN
1360,1361,88,21,4,6,3,1:46.276,NaN,NaN
1361,1362,88,32,9,15,4,1:46.464,NaN,NaN
1362,1363,88,11,16,4,5,1:46.841,NaN,NaN
1363,1364,88,14,9,14,6,1:46.892,NaN,NaN
1364,1365,88,17,3,7,7,1:47.233,NaN,NaN
1365,1366,88,35,15,11,8,1:47.440,NaN,NaN
1366,1367,88,22,6,2,9,1:48.248,NaN,NaN
1367,1368,88,13,15,12,10,1:48.278,NaN,NaN


In [55]:
df_races_weather_all_clean[df_races_weather_all_clean['raceId'] == 88]

,Unnamed: 0,raceId,year,circuitId,name,date,time,url,quali_date,quali_time,...,lng,race_temp,race_precip,race_humidity,race_condition,quali_temp,quali_precip,quali_humidity,quali_condition,race_weather_wiki
469,470,88,2005,22,Japanese Grand Prix,2005-10-09,14:00:00,http://en.wikipedia.org/wiki/2005_Japanese_Gra...,NaN,NaN,...,136.541,23.233333,0.0,50.78,Clear,21.333333,0.0,91.426667,Clear,Weather\nPartially cloudy and dry with tempera...


In [56]:
url= 'https://www.formula1.com/en/results.html/1995/races/637/australia/qualifying-0.html'
page = requests.get(url)
df_q3 = pd.read_html(page.content)[0]

url= 'https://www.formula1.com/en/results.html/1995/races/637/australia/qualifying-1.html'
page = requests.get(url)
df_q1 = pd.read_html(page.content)[0]
url= 'https://www.formula1.com/en/results.html/1995/races/637/australia/qualifying-2.html'
page = requests.get(url)
df_q2 = pd.read_html(page.content)[0]
# %%
df_q3.drop(columns=['Unnamed: 0','Unnamed: 7'],axis=0,inplace=True)
df_q2.drop(columns=['Unnamed: 0','Unnamed: 7'],axis=0,inplace=True)
df_q1.drop(columns=['Unnamed: 0','Unnamed: 7'],axis=0,inplace=True)

In [57]:
df_qualifying_clean[df_qualifying_clean['raceId']==256]

,qualifyId,raceId,driverId,constructorId,number,position,q1,q2,q3
2568,2570,256,71,3,5,1,NaN,NaN,NaN
2569,2571,256,14,3,6,2,NaN,NaN,NaN
2570,2572,256,30,22,1,3,NaN,NaN,NaN
2571,2573,256,77,6,28,4,NaN,NaN,NaN
2572,2574,256,55,6,27,5,NaN,NaN,NaN
2573,2575,256,49,15,30,6,NaN,NaN,NaN
2574,2576,256,22,17,14,7,NaN,NaN,NaN
2575,2577,256,65,22,2,8,NaN,NaN,NaN
2576,2578,256,56,17,15,9,NaN,NaN,NaN
2577,2579,256,87,1,7,10,NaN,NaN,NaN


In [58]:
q1_times = df_qualifying_clean.loc[df_qualifying_clean['raceId']==256,['q1','number']].merge(df_q1[['Time','No']],left_on='number',right_on='No')['Time']
df_qualifying_clean.loc[df_qualifying_clean['raceId']==256,'q1'] = q1_times.values

In [59]:
q2_times = df_qualifying_clean[df_qualifying_clean['raceId']==256].merge(df_q2[['Time','No']],left_on='number',right_on='No',how='left')['Time']
df_qualifying_clean.loc[df_qualifying_clean['raceId']==256,'q2'] = q2_times.values


In [60]:
q3_times = df_qualifying_clean[df_qualifying_clean['raceId']==256].merge(df_q3[['Time','No']],left_on='number',right_on='No',how='left')['Time']
df_qualifying_clean.loc[df_qualifying_clean['raceId']==256,'q3'] = q3_times.values

In [61]:
df_qualifying_clean[df_qualifying_clean['raceId']==256]

,qualifyId,raceId,driverId,constructorId,number,position,q1,q2,q3
2568,2570,256,71,3,5,1,1:15.505,1:15.988,1:15.505
2569,2571,256,14,3,6,2,1:15.628,1:15.792,1:15.628
2570,2572,256,30,22,1,3,1:16.039,1:15.839,1:15.839
2571,2573,256,77,6,28,4,1:15.932,1:16.994,1:15.932
2572,2574,256,55,6,27,5,32:21.400,1:16.305,1:16.305
2573,2575,256,49,15,30,6,1:16.837,1:16.647,1:16.647
2574,2576,256,22,17,14,7,1:16.725,1:16.971,1:16.725
2575,2577,256,65,22,2,8,1:17.289,1:16.950,1:16.950
2576,2578,256,56,17,15,9,1:17.197,1:17.116,1:17.116
2577,2579,256,87,1,7,10,1:17.348,1:17.721,1:17.348


In [62]:
df_qualifying_clean.drop(columns=['number'],inplace=True)

Clean up races and weather data

In [63]:
df_races_clean = f1_data['df_races'].copy()
df_races_weather_clean = f1_data['df_races_weather_all'].copy()

In [64]:
cols=['raceId','fp1_date', 'fp1_time', 'fp2_date', 'fp2_time', 'fp3_date', 'fp3_time']
df_races_weather_clean= df_races_weather_clean.merge(df_races_clean[cols],left_on='raceId',right_on='raceId')
df_races_weather_clean.head()

,Unnamed: 0,raceId,year,circuitId,name,date,time,url,quali_date,quali_time,...,quali_precip,quali_humidity,quali_condition,race_weather_wiki,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time
0,0,1,2009,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...,NaN,NaN,...,0.000000,88.042500,Clear,Weather\nSunny with temperatures reaching up t...,NaN,NaN,NaN,NaN,NaN,NaN
1,1,18,2008,1,Australian Grand Prix,2008-03-16,04:30:00,http://en.wikipedia.org/wiki/2008_Australian_G...,NaN,NaN,...,0.000000,90.102500,Partially cloudy,"Weather\nClear, dry, air temperature of 39 °C ...",NaN,NaN,NaN,NaN,NaN,NaN
2,2,36,2007,1,Australian Grand Prix,2007-03-18,03:00:00,http://en.wikipedia.org/wiki/2007_Australian_G...,NaN,NaN,...,0.000000,82.990000,Clear,Weather\nClear with temperatures reaching up t...,NaN,NaN,NaN,NaN,NaN,NaN
3,3,55,2006,1,Australian Grand Prix,2006-04-02,14:00:00,http://en.wikipedia.org/wiki/2006_Australian_G...,NaN,NaN,...,0.090000,49.036667,Partially cloudy,Weather\nCloudy with sunny breaks. Air tempera...,NaN,NaN,NaN,NaN,NaN,NaN
4,4,71,2005,1,Australian Grand Prix,2005-03-06,14:00:00,http://en.wikipedia.org/wiki/2005_Australian_G...,NaN,NaN,...,1.586667,58.243333,Partially cloudy,"Weather\nCool, partly cloudy",NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
df_races_weather_clean = data_cleaning.weather_flag(df_races_weather_clean,col_in='race_weather_wiki',col_out='race_condition_wiki')

In [66]:
df_races_weather_clean = data_cleaning.weather_flag(df_races_weather_clean,col_in='quali_condition',col_out='quali_condition_clean')

In [67]:
df_races_weather_clean.head()


,Unnamed: 0,raceId,year,circuitId,name,date,time,url,quali_date,quali_time,...,quali_condition,race_weather_wiki,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,race_condition_wiki,quali_condition_clean
0,0,1,2009,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...,NaN,NaN,...,Clear,Sunny with,NaN,NaN,NaN,NaN,NaN,NaN,dry,dry
1,1,18,2008,1,Australian Grand Prix,2008-03-16,04:30:00,http://en.wikipedia.org/wiki/2008_Australian_G...,NaN,NaN,...,Partially cloudy,"Clear, dry,",NaN,NaN,NaN,NaN,NaN,NaN,dry,dry
2,2,36,2007,1,Australian Grand Prix,2007-03-18,03:00:00,http://en.wikipedia.org/wiki/2007_Australian_G...,NaN,NaN,...,Clear,Clear with,NaN,NaN,NaN,NaN,NaN,NaN,dry,dry
3,3,55,2006,1,Australian Grand Prix,2006-04-02,14:00:00,http://en.wikipedia.org/wiki/2006_Australian_G...,NaN,NaN,...,Partially cloudy,Cloudy with,NaN,NaN,NaN,NaN,NaN,NaN,dry,dry
4,4,71,2005,1,Australian Grand Prix,2005-03-06,14:00:00,http://en.wikipedia.org/wiki/2005_Australian_G...,NaN,NaN,...,Partially cloudy,"Cool, partly",NaN,NaN,NaN,NaN,NaN,NaN,dry,dry


Keep the complete race wiki weather database, drop the weather api conditions.

Keep the cleaned quali condition, drop the original

In [68]:
df_races_weather_clean.drop(columns=['race_weather_wiki','race_condition','quali_condition', 'Unnamed: 0'],axis=0,inplace=True)

In [69]:
df_races_weather_clean.race_condition_wiki.value_counts()

dry    967
wet    106
Name: race_condition_wiki, dtype: int64

In [70]:
df_races_weather_clean.quali_condition_clean.value_counts()

dry    672
wet    194
Name: quali_condition_clean, dtype: int64

In [71]:
df_results_clean=f1_data['df_results'].copy()

In [72]:
df_results_clean[df_results_clean.isna().any(axis=1)]

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
5,6,18,6,3,8,13,6,6,6,3.0,57,NaN,NaN,50,14,1:29.639,212.974,11
6,7,18,7,5,14,17,7,7,7,2.0,55,NaN,NaN,22,12,1:29.534,213.224,5
7,8,18,8,6,1,15,8,8,8,1.0,53,NaN,NaN,20,4,1:27.903,217.180,5
8,9,18,9,2,4,2,NaN,R,9,0.0,47,NaN,NaN,15,9,1:28.753,215.100,4
9,10,18,10,7,12,18,NaN,R,10,0.0,43,NaN,NaN,23,13,1:29.558,213.166,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25715,25721,1089,825,210,20,16,16,16,16,0.0,52,NaN,NaN,37,16,1:26.798,240.268,11
25716,25722,1089,817,1,3,4,NaN,R,17,0.0,45,NaN,NaN,39,12,1:26.603,240.809,44
25717,25723,1089,840,117,18,12,NaN,R,18,0.0,39,NaN,NaN,34,19,1:27.467,238.430,5
25718,25724,1089,4,214,14,6,NaN,R,19,0.0,31,NaN,NaN,7,18,1:27.203,239.152,34


In [73]:
(df_results_clean.time.isna()).sum()/len(df_results_clean)*100

72.75272161741835

In [74]:
(df_results_clean.position.isna()).sum()/len(df_results_clean)*100

42.12674961119751

PositionText, position and position Order replicate themselves for now will keep PositionOrder and positionText as can decided to change Retired to joint last if I would like later. Number is the driver number and is in the driver database. Time is repitition of milliseconds, so can drop time

In [75]:
df_results_clean.drop(columns=['number','position','time'],axis=0,inplace=True)

In [76]:
df_seasons_clean=f1_data['df_seasons'].copy()
df_seasons_clean.head()

,year,url
0,2009,http://en.wikipedia.org/wiki/2009_Formula_One_...
1,2008,http://en.wikipedia.org/wiki/2008_Formula_One_...
2,2007,http://en.wikipedia.org/wiki/2007_Formula_One_...
3,2006,http://en.wikipedia.org/wiki/2006_Formula_One_...
4,2005,http://en.wikipedia.org/wiki/2005_Formula_One_...


In [77]:
df_sprint_results_clean=f1_data['df_sprint_results'].copy()
df_sprint_results_clean.head()

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,fastestLapTime,statusId
0,1,1061,830,9,33,2,1,1,1,3,17,25:38.426,1538426,14,1:30.013,1
1,2,1061,1,131,44,1,2,2,2,2,17,+1.430,1539856,17,1:29.937,1
2,3,1061,822,131,77,3,3,3,3,1,17,+7.502,1545928,17,1:29.958,1
3,4,1061,844,6,16,4,4,4,4,0,17,+11.278,1549704,16,1:30.163,1
4,5,1061,846,1,4,6,5,5,5,0,17,+24.111,1562537,16,1:30.566,1


Number is driver code in the driver database, same as results position Order and position Text hold the relevant position data. Time and miliseconds repeat each other. Drop time

In [78]:
df_sprint_results_clean.drop(columns=['position','time','number'],inplace=True)

In [79]:
df_status_clean=f1_data['df_status'].copy()
df_status_clean.head()

,statusId,status
0,1,Finished
1,2,Disqualified
2,3,Accident
3,4,Collision
4,5,Engine


## Merge dataframe into one complete dataframe minus laptime and pitstops which will check with fast f1 dataset first

In [81]:
f1_data=f1_ultils.read_ergast_files('./data/raw')
data_cleaning.clean_ergast(f1_data,save=False)

,raceId,year,circuitId,name_x,date,time,quali_date,quali_time,sprint_date,sprint_time,...,position_constructors,wins_constructors,constructorResultsId,points_constructor_ind_races,driverRef,number,dob,nationality_drivers,constructorRef,nationality_constructors
0,1,2009,1,Australian Grand Prix,2009-03-29,06:00:00,NaN,NaN,NaN,NaN,...,1.0,1.0,3823.0,18.0,button,22,1980-01-19,UK,brawn,UK
1,1,2009,1,Australian Grand Prix,2009-03-29,06:00:00,NaN,NaN,NaN,NaN,...,1.0,1.0,3823.0,18.0,barrichello,\N,1972-05-23,Brazil,brawn,UK
2,1,2009,1,Australian Grand Prix,2009-03-29,06:00:00,NaN,NaN,NaN,NaN,...,8.0,0.0,3831.0,0.0,vettel,5,1987-07-03,Germany,red_bull,Austria
3,1,2009,1,Australian Grand Prix,2009-03-29,06:00:00,NaN,NaN,NaN,NaN,...,7.0,0.0,3830.0,0.0,kubica,88,1984-12-07,Poland,bmw_sauber,Germany
4,1,2009,1,Australian Grand Prix,2009-03-29,06:00:00,NaN,NaN,NaN,NaN,...,4.0,0.0,3827.0,3.0,rosberg,6,1985-06-27,Germany,williams,UK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9450,1078,2022,79,Miami Grand Prix,2022-05-08,19:30:00,2022-05-07,20:00:00,NaN,NaN,...,8.0,0.0,16499.0,0.0,kevin_magnussen,20,1992-10-05,Denmark,haas,USA
9451,1078,2022,79,Miami Grand Prix,2022-05-08,19:30:00,2022-05-07,20:00:00,NaN,NaN,...,5.0,0.0,16493.0,6.0,zhou,24,1999-05-30,China,alfa,Switzerland
9452,1078,2022,79,Miami Grand Prix,2022-05-08,19:30:00,2022-05-07,20:00:00,NaN,NaN,...,10.0,0.0,16495.0,2.0,albon,23,1996-03-23,Thailand,williams,UK
9453,1078,2022,79,Miami Grand Prix,2022-05-08,19:30:00,2022-05-07,20:00:00,NaN,NaN,...,10.0,0.0,16495.0,2.0,latifi,6,1995-06-29,Canada,williams,UK
